In [1]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
import psycopg2

%matplotlib inline

con = psycopg2.connect (user = 'postgres',
                password = 'postgres',
                port = '5433',
                host = 'localhost',                
                database = 'inicudb')

cur  = con.cursor()

In [2]:
uhidPatient = "RSHI.0000012088"
caseType = 'Death'
conditionCase = "(dischargestatus = 'Death' or dischargestatus = 'LAMA' )"
#uhidPatient = "RSHI.0000012984"
#caseType = 'Discharge'
#conditionCase = "(dischargestatus = 'Discharge')"
seperator = '_'

In [3]:
#base = pd.read_csv("/Users/harpreet/Videos/data_22_june.csv",  nrows=999999)

In [4]:
#ds = base[~base.starttime.notna()]

In [5]:
#ds.hour_series.head()

In [6]:
#base[~base.hour_series.notna()]

In [7]:
#death_base = base[base.dischargestatus == 1]

In [8]:
#discharge_base = base[base.dischargestatus == 0]

In [9]:
#death_base.uhid.unique()


In [10]:
#discharge_base.uhid.unique()

In [11]:
#dea = death_base.groupby('uhid').head(1)
#dis = discharge_base.groupby('uhid').head(1)

In [12]:
#checkingUHIDData = death_base[death_base.uhid==uhidPatient]

In [13]:
#checkingUHIDData.to_csv(uhidPatient+".csv")

In [14]:
#print(dea.gestationweekbylmp, dea.birthweight,dea.uhid)

In [15]:
#print(dea.gestationweekbylmp, dea.birthweight,dea.uhid)

In [16]:
#lesstha26 = dis[(dis.gestationweekbylmp <= 26) & (dis.birthweight<=1500)]
#between26and28 = dis[(dis.gestationweekbylmp > 26) & (dis.gestationweekbylmp <= 28) & (dis.birthweight<=1500)]
#between28and32 = dis[(dis.gestationweekbylmp > 28) & (dis.gestationweekbylmp <= 32) & (dis.birthweight<=1500)]
#greaterthan32 = dis[(dis.gestationweekbylmp > 32) & (dis.birthweight<=3000) & (dis.birthweight>=2500)]

In [17]:
#(lesstha26.birthweight)




In [18]:
#(between26and28.birthweight)

In [19]:
#(between28and32.birthweight)

In [20]:
#len(greaterthan32)

In [21]:
cur10 = con.cursor()
cur10.execute("SELECT t1.uhid,t1.dateofbirth,t1.timeofbirth,t1.timeofadmission,t1.dateofadmission,t1.gender,t1.birthweight,t1.birthlength,t1.birthheadcircumference,t1.inout_patient_status,t1.gestationweekbylmp,t1.gestationdaysbylmp,t1.baby_type,t1.central_temp,t1.dischargeddate,t2.apgar_onemin,t2.apgar_fivemin,t2.apgar_tenmin,t3.motherage,t4.conception_type,t4.mode_of_delivery,t5.steroidname,t5.numberofdose,t1.dischargestatus  FROM apollo.baby_detail AS t1 LEFT JOIN apollo.birth_to_nicu AS t2 ON t1.uhid=t2.uhid LEFT JOIN apollo.parent_detail AS t3 ON t1.uhid=t3.uhid LEFT JOIN apollo.antenatal_history_detail AS t4 ON t1.uhid=t4.uhid LEFT JOIN apollo.antenatal_steroid_detail AS t5 ON t1.uhid=t5.uhid where timeofadmission is not null and timeofbirth is not null and t1.uhid = '"+uhidPatient+"';")
cols10 = list(map(lambda x: x[0], cur10.description))
fixed = pd.DataFrame(cur10.fetchall(),columns=cols10)

In [22]:
len(fixed)

1

In [23]:
s1 = set(fixed.uhid.unique())

In [24]:
cur11 = con.cursor()
cur11.execute("SELECT DISTINCT(uhid) FROM apollo.baby_detail WHERE dateofadmission >= '2018-07-01' AND dateofadmission <= '2020-06-25' and UHID IN  (select distinct(uhid) from apollo.babyfeed_detail where uhid in  ( select distinct(uhid) from apollo.baby_visit where uhid in (select  distinct(uhid) from apollo.nursing_vitalparameters where uhid in ( select distinct(uhid) from apollo.device_monitor_detail UNION select distinct(uhid) from apollo.device_monitor_detail_dump)))) and "+conditionCase+" and isreadmitted is not true and gestationweekbylmp is not null and birthweight is not null and uhid = '"+uhidPatient+"';")
cols11 = list(map(lambda x: x[0], cur11.description))
uhids = pd.DataFrame(cur11.fetchall(),columns=cols11)

In [25]:
print(len(uhids.uhid.unique()))

1


In [26]:
s2 = set(uhids.uhid.unique())

In [27]:
l = list(s1 - s1.intersection(s2))

In [28]:
len(uhids)

1

In [29]:
fixed.set_index('uhid',inplace=True)

In [30]:
df = fixed.drop(l)

In [31]:
len(df.drop_duplicates())

1

In [32]:
df.reset_index(inplace=True)

In [33]:
len(df.uhid.unique())

1

In [34]:
#dates_detail = ds.copy()

In [35]:
dates_detail = df.copy()

In [36]:
dates_detail.head()

,uhid,dateofbirth,timeofbirth,timeofadmission,dateofadmission,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,dischargeddate,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus
0,RSHI.0000012088,2018-07-10,"08,57,AM","12,30,PM",2018-07-10,Female,1408.0,None,None,In Born,...,2018-08-01 20:40:04.099,7,8,None,31,ivf,LSCS,None,None,Death


In [37]:
def second_addition(x):
    try:
        if x.split(",")[2] == 'PM':
            if x.split(",")[0] != '12':
                return (12 + float(x.split(",")[0]) + float(x.split(",")[1])/60)*3600
            else:
                return (float(x.split(",")[0]) + float(x.split(",")[1])/60)*3600
        else:
            return (float(x.split(",")[0]) + float(x.split(",")[1])/60)*3600
    except:
        
        if x.split(" ")[1] == 'PM':
            if (x.split(" ")[0]).split(":")[0] != '12':
                return (12 + float((x.split(" ")[0]).split(":")[0]) + float((x.split(" ")[0]).split(":")[1])/60)*3600
            else:
                return (float((x.split(" ")[0]).split(":")[0]) + float((x.split(" ")[0]).split(":")[1])/60)*3600
        else:
            return (float((x.split(" ")[0]).split(":")[0]) + float((x.split(" ")[0]).split(":")[1])/60)*3600
    
        

In [38]:
def split_date(x):
    try:
        t=str(x)
        return t.split(":")[0] + ":" + t.split(":")[1]
    except:
        pass

def split_hour(x):
    try:
        t=str(x)
        return t.split(":")[0]
    except:
        pass
    
def split_date_1(x):
    try:
        t=str(x)
        return str(t.split(" ")[0])
    except:
        pass
    

In [39]:
dates_detail['add_seconds'] = dates_detail['timeofbirth'].apply(second_addition)
dates_detail['add_seconds_admission'] = dates_detail['timeofadmission'].apply(second_addition)

In [40]:
from datetime import timedelta

In [41]:
def actual_birthdate(x,y):
    try:
        return pd.to_datetime(x) + timedelta(seconds=y)
    except:
        pass

In [42]:
dates_detail['actual_DOB'] = dates_detail.apply(lambda x: actual_birthdate(x['dateofbirth'], x['add_seconds']), axis=1)

In [43]:
dates_detail['actual_DOA'] = dates_detail.apply(lambda x: actual_birthdate(x['dateofadmission'], x['add_seconds_admission']), axis=1)

In [44]:
dd = dates_detail[['actual_DOB','uhid','dischargeddate','actual_DOA','gender','birthweight','birthlength','birthheadcircumference','inout_patient_status'
,'gestationweekbylmp','gestationdaysbylmp','baby_type','central_temp','apgar_onemin',
'apgar_fivemin','apgar_tenmin','motherage','conception_type','mode_of_delivery','steroidname',
'numberofdose','dischargestatus','birthlength','birthheadcircumference','inout_patient_status'
,'gestationweekbylmp','gestationdaysbylmp','baby_type','central_temp','apgar_onemin',
'apgar_fivemin','apgar_tenmin','motherage','conception_type','mode_of_delivery','steroidname',
'numberofdose']]

In [45]:
dd.dropna(subset=['dischargeddate'],inplace=True)

/Users/harpreet/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
dd['los'] = dates_detail['dischargeddate'] - dates_detail['actual_DOA']

/Users/harpreet/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [47]:
def day(x):
    return x.total_seconds()/86400

In [48]:
dd['day_1'] = dd['los'].apply(day)

/Users/harpreet/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
dd = dd[dd['day_1']>=0]

In [50]:
dd.sort_values(by = ['actual_DOA'],inplace=True)

In [51]:
dd.drop_duplicates(subset=['uhid'],keep='first',inplace=True)

In [52]:
dd

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,...,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1
0,2018-07-10 08:57:00,RSHI.0000012088,2018-08-01 20:40:04.099,2018-07-10 12:30:00,Female,1408.0,None,None,In Born,28,...,7,8,None,31,ivf,LSCS,None,None,22 days 08:10:04.099000,22.340325


In [53]:
dt = pd.DataFrame()

In [54]:
dt

""


In [55]:
dd

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,...,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1
0,2018-07-10 08:57:00,RSHI.0000012088,2018-08-01 20:40:04.099,2018-07-10 12:30:00,Female,1408.0,None,None,In Born,28,...,7,8,None,31,ivf,LSCS,None,None,22 days 08:10:04.099000,22.340325


In [56]:
dd.columns

Index(['actual_DOB', 'uhid', 'dischargeddate', 'actual_DOA', 'gender',
       'birthweight', 'birthlength', 'birthheadcircumference',
       'inout_patient_status', 'gestationweekbylmp', 'gestationdaysbylmp',
       'baby_type', 'central_temp', 'apgar_onemin', 'apgar_fivemin',
       'apgar_tenmin', 'motherage', 'conception_type', 'mode_of_delivery',
       'steroidname', 'numberofdose', 'dischargestatus', 'birthlength',
       'birthheadcircumference', 'inout_patient_status', 'gestationweekbylmp',
       'gestationdaysbylmp', 'baby_type', 'central_temp', 'apgar_onemin',
       'apgar_fivemin', 'apgar_tenmin', 'motherage', 'conception_type',
       'mode_of_delivery', 'steroidname', 'numberofdose', 'los', 'day_1'],
      dtype='object')

In [57]:
dd.dischargeddate

0   2018-08-01 20:40:04.099
Name: dischargeddate, dtype: datetime64[ns]

In [58]:
dd.dischargeddate.iloc[0]

Timestamp('2018-08-01 20:40:04.099000')

In [59]:
dd.actual_DOA.iloc[0]

Timestamp('2018-07-10 12:30:00')

In [60]:
x = dd[dd['uhid']==uhidPatient]

In [61]:
x

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,...,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1
0,2018-07-10 08:57:00,RSHI.0000012088,2018-08-01 20:40:04.099,2018-07-10 12:30:00,Female,1408.0,None,None,In Born,28,...,7,8,None,31,ivf,LSCS,None,None,22 days 08:10:04.099000,22.340325


In [62]:
(x.dischargeddate.iloc[0] - x.actual_DOA.iloc[0]).total_seconds()

1930204.099

In [ ]:
import math
for i in dd.uhid.unique():
    x = dd[dd['uhid']==i]
    time_s = []
    n = math.ceil((x.dischargeddate.iloc[0] - x.actual_DOA.iloc[0]).total_seconds()/60)
    try:
        x = pd.concat([x]*int(n))

        for i in range(0,len(x)):
            #print(i)
            time_s.append(x['actual_DOA'].iloc[i] + timedelta(seconds = i*60))
            #t['hour_series'].loc[i] = t['actual_DOA'].iloc[i] + timedelta(hours = i)

        x['hour_series'] = time_s
    except Exception as e:
        print(i,n, e)
        continue
    dt = dt.append(x,ignore_index=True)


In [ ]:
len(dt)

In [ ]:
dt

In [ ]:
dt

In [ ]:
dt.drop_duplicates(subset=['uhid','hour_series'],inplace=True)

In [ ]:
dt

In [ ]:
cur1 = con.cursor()
cur1.execute("select distinct(b.uhid),l.conception_type, b.gender, b.dateofadmission, b.dischargestatus,b.birthweight,b.weight_galevel,b.weight_centile,b.birthlength,b.birthheadcircumference, b.inout_patient_status, b.gestationweekbylmp, b.gestationdaysbylmp,round( CAST((b.gestationweekbylmp + b.gestationdaysbylmp/7::float) as numeric),2) as Gestation,b.dischargeddate, b.admissionweight, b.baby_type,b.baby_number, b.branchname ,DATE_PART('day',b.dischargeddate - b.dateofadmission) as LOS,c.apgar_onemin, c.apgar_fivemin,c.apgar_tenmin, c.resuscitation,d.isantenatalsteroidgiven, d.mode_of_delivery,z.motherage,e.jaundicestatus as JAUNDICE,  f.eventstatus as SEPSIS,f.progressnotes, g.eventstatus as RDS, g.progressnotes,y.eventstatus as ASPHYXIA,y.progressnotes from apollo.baby_detail as b left join apollo.birth_to_nicu as c on b.uhid = c.uhid and b.episodeid = c.episodeid left join apollo.parent_detail as z on b.uhid = z.uhid and b.episodeid = z.episodeid left join apollo.antenatal_history_detail as d on b.uhid = d.uhid and b.episodeid = d.episodeid left join apollo.sa_jaundice AS e ON b.uhid = e.uhid and e.jaundicestatus = 'Yes' and e.phototherapyvalue='Start' left join apollo.sa_infection_sepsis AS f ON b.uhid = f.uhid and f.eventstatus = 'yes' and f.episode_number = 1 left join apollo.sa_cns_asphyxia AS y ON b.uhid = y.uhid and y.eventstatus = 'yes' and y.episode_number = 1 left join apollo.antenatal_history_detail as l on b.uhid=l.uhid left join apollo.sa_resp_rds AS g ON b.uhid = g.uhid and g.eventstatus = 'Yes' and g.episode_number = 1 and g.uhid IN (select distinct(h.uhid) from apollo.respsupport AS h where h.eventname='Respiratory Distress' and (h.rs_vent_type ='Mechanical Ventilation' OR h.rs_vent_type ='HFO') UNION select distinct(i.uhid) from apollo.sa_resp_rds AS i where i.sufactantname is not null) and b.uhid = '"+uhidPatient+ "' order by b.dateofadmission")
cols1 = list(map(lambda x: x[0], cur1.description))
ds = pd.DataFrame(cur1.fetchall(),columns=cols1)
#ds.sort_values('modificationtime',inplace=True)

In [ ]:
len(dt.uhid.unique())

In [ ]:
data = ds.copy()

In [ ]:
dt

In [ ]:
ids = dt.uhid.unique()

In [ ]:
len(dt)

In [ ]:
data.drop_duplicates('uhid',keep='first',inplace=True)

In [ ]:
df = pd.DataFrame(columns=data.columns)

In [ ]:
for i in ids:
    x = data[data['uhid']==i]
    df = df.append(x,ignore_index=True)

In [ ]:
#d = pd.read_csv('baseline.csv')

In [ ]:
#len(d)

In [ ]:
#s = set(d['uhid'].unique())

In [ ]:
pd.set_option('display.max_columns',100)

In [ ]:
#s1 = set(ids)

In [ ]:
ds = pd.DataFrame()

In [ ]:
ds = dt.copy()

In [ ]:
#ds.to_csv('test_baseline.csv')

In [ ]:
uhid = ds.uhid.unique()

In [ ]:
dt = pd.DataFrame()

In [ ]:
for i in uhid:
    x = dd[dd['uhid']==i]
    time_s = []
    n = math.ceil((x.dischargeddate.iloc[0] - x.actual_DOA.iloc[0]).total_seconds()/60)
    try:
        x = pd.concat([x]*int(n))
        for i in range(0,len(x)):
            #print(i)
            time_s.append(x['actual_DOA'].iloc[i] + timedelta(seconds = i*60))
            #t['hour_series'].loc[i] = t['actual_DOA'].iloc[i] + timedelta(hours = i)

        x['hour_series'] = time_s
    except Exception as inst:
        print(inst)
        continue
    dt = dt.append(x,ignore_index=True)


In [ ]:
dd['uhid']

In [ ]:
len(dt)

In [ ]:
x

In [ ]:
uhid

In [ ]:
dt['hour_series'].apply(split_hour)

In [ ]:
dt['hour'] = dt['hour_series'].apply(split_hour)

In [ ]:
dt['day'] = dt['hour_series'].apply(split_date_1)

In [ ]:
len(dt.uhid.unique())

In [ ]:
#output
cur9= con.cursor()
cur9.execute("SELECT t1.uhid,t1.abdomen_girth,t1.urine,t1.stool,t1.stool_passed, t1.entry_timestamp FROM apollo.nursing_intake_output AS t1 where t1.uhid = '"+uhidPatient+"'")
cols9 = list(map(lambda x: x[0], cur9.description))
output = pd.DataFrame(cur9.fetchall(),columns=cols9)

In [ ]:
dt

In [ ]:
output_d = pd.DataFrame()

In [ ]:
for i in uhid:
    x = output[output['uhid']==i]
    output_d = output_d.append(x,ignore_index=True)

In [ ]:
output_d.entry_timestamp = output_d.entry_timestamp + timedelta(seconds=19800)

In [ ]:
output_d

In [ ]:
output_d['hour'] = output_d.entry_timestamp.apply(split_hour)
output_d['day'] = output_d.entry_timestamp.apply(split_date_1)

In [ ]:
output_d

In [ ]:
output_d.sort_values(by=['uhid','entry_timestamp'],inplace=True)

In [ ]:
len(output_d)

In [ ]:
test = output_d.drop_duplicates(subset=['uhid','hour'],keep='last')

In [ ]:
test.head()

In [ ]:
dt.head()

In [ ]:
dh = pd.merge(dt,test,on=['uhid','hour'],how='left')

In [ ]:
dh

In [ ]:
#dh.sort_values(by=['uhid','entry_timestamp'],inplace=True)

In [ ]:
dh.sort_values('hour_series')

In [ ]:
dg = pd.DataFrame()

In [ ]:
dh

In [ ]:
len(dh)

In [ ]:
x.columns

In [ ]:
x.urine

In [ ]:
for i in uhid:
    x = dh[dh['uhid']==i]
    #x.dropna(subset=['urine'],inplace=True)
    x['time_divide'] = 0
    print(len(x))
    startTime = None
    for j in range(len(x)):
        try:
            if j == 0:
                x.loc[j,'time_divide'] = 0.0                
                startTime = x.actual_DOA.iloc[j]
            else:
                if(x.urine.iloc[j] != None):
                    urine_value = float(x.urine.iloc[j])
                    if(~math.isnan(urine_value) and (urine_value >0)):
                        x.loc[j,'time_divide'] = (pd.to_datetime(str(x.entry_timestamp.iloc[j]).split("+")[0]) - startTime).total_seconds()
                        startTime = pd.to_datetime(str(x.entry_timestamp.iloc[j]).split("+")[0])
            #print(x.loc[j,'time_divide'])
        except Exception as e:
            print(j,'Exception is',e)
            continue
    dg = dg.append(x,ignore_index=True)    

In [ ]:
dg.time_divide.unique()

In [ ]:
dg

In [ ]:
dg = dg.sort_values('hour_series')

In [ ]:
dg

In [ ]:
dg.time_divide.unique()

In [ ]:
dg = dg.sort_values('hour_series')

In [ ]:
dg

In [ ]:
dg.columns


In [ ]:
dg.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_new_1.csv')

In [ ]:
dg